# Custom Chatbot Notebook

An OpenAI client is initialised by using environment variables and a tokenizer is set up for a specific model (`gpt-4o-mini-2024-07-18`). Also,the necessary libraries and custom utility functions are imported.

In [65]:
import pandas as pd
import os
from pathlib import Path
from dotenv import load_dotenv
import tiktoken

# Custom Functions
from fncs.utilities import (
    create_openai_client,
    response_generator,
    prompt_builder,
    calculate_total_cost
    )
from fncs.retrieval import (
    get_embedding,
    search_text,
    control_chunk_context
    )

# Load environment vars:
load_dotenv()
base_url_voc = os.getenv("OPENAI_BASE_VOC")
api_key_voc = os.getenv("OPENAI_API_VOC")
# Deployment model names
chat_name = 'gpt-4o' # 'gpt-4o-mini-2024-07-18' # 'gpt-4o-mini'
emb_name = 'text-embedding-3-large'
# Initialising OpenAI client
openai_client = create_openai_client(api_key= api_key_voc, base_url= base_url_voc)
tokenizer = tiktoken.encoding_for_model(chat_name)

### Loading dataset

In [66]:
proj_dir = Path(os.getcwd())
df = pd.read_csv(proj_dir / "data" / "2023_fashion_trends_embeddings.csv")
df.head(3)

,text,embeddings
0,Title: 7 Fashion Trends That Will Take Over 20...,"[-0.06084602698683739, -0.00787690281867981, -..."
1,Title: 7 Fashion Trends That Will Take Over 20...,"[-0.06700262427330017, -0.014003804884850979, ..."
2,Title: 7 Fashion Trends That Will Take Over 20...,"[-0.05102064833045006, -0.00858586560934782, -..."


The embeddings are stored as text/string in the DataFrame and need to be converted to lists/arrays

In [67]:
import ast
# Converting the string representations of embeddings to actual lists
df['embeddings'] = df['embeddings'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

Checking transformation

In [68]:
type(df[['embeddings']].iloc[0].values[0])

list

### Calculating Cosine Distances based on query


Below I create a query string about fashion trends in 2023. Then, by using the `get_embedding` function, the embeddings of the query are generated, by passing the query, OpenAI client, and embedding model as inputs.

In [69]:
query = "What is the most popular fashion trend about pants in 2023?"
query_emb = get_embedding(text=query, client = openai_client, model=emb_name)

The DataFrame `df` is sorted based on the cosine distance between the query embedding (`query_emb`) and the embeddings in the DataFrame using the `search_text` function, and stores the result in `df_sorted`.

In [70]:
df_sorted = search_text(df=df, embs_query=query_emb, cosine='distance')

In [71]:
df_sorted.head()

,text,embeddings,distance
1,Title: 7 Fashion Trends That Will Take Over 20...,"[-0.06700262427330017, -0.014003804884850979, ...",0.273721
3,Title: 7 Fashion Trends That Will Take Over 20...,"[-0.05067730322480202, -0.02512504905462265, -...",0.307084
58,Title: Spring/Summer 2023 Fashion Trends: 21 E...,"[-0.03485928103327751, -0.015784457325935364, ...",0.309776
44,Title: Spring/Summer 2023 Fashion Trends: 21 E...,"[-0.04672637954354286, -0.03269721940159798, -...",0.310095
19,Title: 9 Spring 2023 Fashion Trends You’ll Wan...,"[-0.04425228014588356, -0.035396404564380646, ...",0.355568


### Prompt Template



Creating the system prompt to be used in the chatbot

In [72]:
system_prompt = "You are an expert fashion trend analyser. Based only on the provided information you must analyse and summarise the trends and provide an accurate answer."

print(f"System Prompt Tokens: {len(tokenizer.encode(system_prompt))}")

System Prompt Tokens: 28


Creating the user prompt to be used in the chatbot

In [73]:
user_prompt = \
"""
***Question: {}

***Context:
<--Start of Context-->
{}
<--End of Context-->

**Instructions:
- Answer based ONLY on the provided context above
- Do not include external knowledge
- Be concise and specific

**Required Format:
1. Answer:
   [Your detailed response here]

2. Key Points:
   • [Bullet point 1]
   • [Bullet point 2]
   • [...]

3. Sources:
   • [Source URL 1]
   • [Source URL 2]

Note: If the answer cannot be determined from the provided context,
state: "Cannot be determined from the given context."
"""
print(f"User Prompt Tokens BEFORE context insertion: {len(tokenizer.encode(user_prompt))}")

User Prompt Tokens BEFORE context insertion: 130


In [74]:
# to be used in performance demonstration later
user_prompt_without_context = \
"""
***Question: {}

**Instructions:
- Be concise and specific

**Required Format:
1. Answer:
   [Your detailed response here]

2. Key Points:
   • [Bullet point 1]
   • [Bullet point 2]
   • [...]

3. Sources:
   • [Source URL 1]
   • [Source URL 2]
"""
print(f"User Prompt Tokens BEFORE context insertion: {len(tokenizer.encode(user_prompt))}")

User Prompt Tokens BEFORE context insertion: 130


#### Apply token controller function ( fnc: control_chunk_context )

The variable `max_token_count` to 1000, serves as a limit for the total number of tokens allowed in a prompt.

In [75]:
#parameter that control the prompt tokens:
max_token_count = 1000

The code below calculates the current token count of the prompts (system and user) and generates a context by selecting data from the sorted DataFrame (`df_sorted`) based on a maximum allowed token limit (`max_token_count`) using the `control_chunk_context` function.

In [76]:
current_token_count = len(tokenizer.encode(user_prompt)) + len(tokenizer.encode(system_prompt))
# Create context from sorted dataframe according to the max token limit
context = control_chunk_context(
    df_sorted,
    current_token_count,
    max_token_count,
    tokenizer = tokenizer
)

 Below, the final `user_prompt` is created by inserting the generated `context` into the prompt template and by formatting it with the query and context.

In [77]:
# prompt template params
context_inprompt = "\n----\n".join(context)
user_prompt_0 = user_prompt.format(query, context_inprompt)

print(user_prompt)


***Question: {}

***Context:
<--Start of Context-->
{}
<--End of Context-->

**Instructions:
- Answer based ONLY on the provided context above
- Do not include external knowledge
- Be concise and specific

**Required Format:
1. Answer:
   [Your detailed response here]

2. Key Points:
   • [Bullet point 1]
   • [Bullet point 2]
   • [...]

3. Sources:
   • [Source URL 1]
   • [Source URL 2]

Note: If the answer cannot be determined from the provided context,
state: "Cannot be determined from the given context."



In [78]:
print(f"User Prompt Tokens AFTER context insertion: {len(tokenizer.encode(user_prompt))}")

User Prompt Tokens AFTER context insertion: 130


## Custom Query Completion

**Finally, the code below generates a final prompt using the `prompt_builder` function by combining the system and user prompts. It then sends the prompt to the OpenAI model (`chat_model`) using the `response_generator` function with specified additional options (e.g., `temperature=0`) to generate an AI response. It also calculates the total cost in EUR based on the API usage (`response_full.usage`) for the specific deployment (`gpt-4o-mini`).**

In [79]:
final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= user_prompt_0)
additional_options = {"temperature": 0.4,}
response, response_full = response_generator(openai_client, chat_model=chat_name, prompts=final_prompt, options=additional_options)
cost_eur = calculate_total_cost(response_usage= response_full.usage,
                                deployment_name= chat_name)
print(f'Query Completion Total Cost is: {cost_eur} eur')

Query Completion Total Cost is: 0.00414349086 eur


In [80]:
print(response)

1. Answer:
   The most popular fashion trend about pants in 2023 is the resurgence and evolution of cargo pants. These are being reimagined with tailored silhouettes, interesting pocket placements, and elevated fabrics, moving beyond traditional khaki and olive colors. Additionally, baggy and wide-leg denim styles are also trending, with a focus on looser fits and floor-grazing lengths.

2. Key Points:
   • Cargo pants are making a comeback with tailored designs and elevated fabrics.
   • Baggy and wide-leg denim styles are popular, emphasizing looser fits.
   • The trend includes diverse pant styles such as pedal pushers, wide-leg, and puddle hemlines.

3. Sources:
   • www.refinery29.com
   • www.whowhatwear.com


In [81]:
print('Total Tokens: ', response_full.usage.total_tokens)
print('Total Completion Tokens: ', response_full.usage.completion_tokens)
print('Total Prompt Tokens: ', response_full.usage.prompt_tokens)

Total Tokens:  1087
Total Completion Tokens:  161
Total Prompt Tokens:  926


## Demonstrating Performance

Below, two questions (queries) are ... ...

### Question 1
**Question**: According to Vogue, what is a new trend presented by Prada on New York Fashion Week?

In [82]:
query_1 = "According to Vogue, what is a new trend presented by Prada on New York Fashion Week?"
max_token_count = 1000

In [83]:
query_emb = get_embedding(text=query_1, client = openai_client, model=emb_name)
df_sorted = search_text(df=df, embs_query=query_emb, cosine='distance')

current_token_count = len(tokenizer.encode(user_prompt)) + len(tokenizer.encode(system_prompt))
# Create context from sorted dataframe according to the max token limit
context = control_chunk_context(chunks_sorted_df=df_sorted,
                                current_token_count=current_token_count,
                                max_token_count=max_token_count,
                                tokenizer = tokenizer)
context_inprompt = "\n----\n".join(context)
user_prompt_1 = user_prompt.format(query_1, context_inprompt)

final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= user_prompt_1)
additional_options = {"temperature": 0,}

response_1_1, response_full_1_1 = \
    response_generator(openai_client, chat_model=chat_name, prompts=final_prompt, options= additional_options)

cost_eur_1_1 = \
    calculate_total_cost(response_usage= response_full.usage, deployment_name= chat_name)
print(f'Query Completion Total Cost is: {cost_eur_1_1} eur')


Query Completion Total Cost is: 0.00414349086 eur


In [84]:
print(response_1_1)

1. Answer:
   Prada presented a trend of "Perfectly Imperfect" fashion at New York Fashion Week, characterized by a sense of "unfinishedness" with features like irregularly dyed prints and slits that appear torn.

2. Key Points:
   • Prada's trend is called "Perfectly Imperfect."
   • It features an "unfinished" look with irregularly dyed prints.
   • The design includes slits that appear as if the garment is torn.

3. Sources:
   • www.vogue.com


In [85]:
final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= query_1) #or use: user_prompt_without_context.format(query_1)
additional_options = {"temperature": 0,}

response_1_2, response_full_1_2 = response_generator(openai_client, chat_model=chat_name, prompts=final_prompt, options=additional_options)
cost_eur_1_2 = calculate_total_cost(response_usage= response_full.usage,
                                deployment_name= chat_name)
print(f'Query Completion Total Cost is: {cost_eur_1_2} eur')

Query Completion Total Cost is: 0.00414349086 eur


In [86]:
print(response_1_2)

Prada presented a new trend at New York Fashion Week that focused on minimalist and utilitarian designs. This trend emphasized clean lines, functional details, and a neutral color palette, showcasing a shift towards simplicity and practicality in fashion.


### Question 2
**Question**:

In [92]:
query_2 = "According to glamour magazine and whowhatwear.com what are the denim fashion trends for the year 2023?"
max_token_count = 1000

In [93]:
query_emb = get_embedding(text=query_2, client = openai_client, model=emb_name)
df_sorted = search_text(df=df, embs_query=query_emb, cosine='distance')

current_token_count = len(tokenizer.encode(user_prompt)) + len(tokenizer.encode(system_prompt))
# Create context from sorted dataframe according to the max token limit
context = control_chunk_context(chunks_sorted_df=df_sorted,
                                current_token_count=current_token_count,
                                max_token_count=max_token_count,
                                tokenizer = tokenizer)
context_inprompt = "\n----\n".join(context)
user_prompt_2 = user_prompt.format(query_2, context_inprompt)

final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= user_prompt_2)
additional_options = {"temperature": 0,}

response_2_1, response_full_2_1 = \
    response_generator(openai_client, chat_model=chat_name, prompts=final_prompt, options= additional_options)

cost_eur_2_1 = \
    calculate_total_cost(response_usage= response_full.usage, deployment_name= chat_name)
print(f'Query Completion Total Cost is: {cost_eur_2_1} eur')

Query Completion Total Cost is: 0.00414349086 eur


In [94]:
print(response_2_1)

1. Answer:
   The denim fashion trends for 2023, according to Glamour Magazine and WhoWhatWear.com, include baggy and looser-fit denim styles, the return of the denim-on-denim look, and the influence of '90s and '00s fashion with items like denim maxi skirts. There is a focus on relaxed silhouettes, with wide-leg and slouchy fits being prominent.

2. Key Points:
   • Baggy and looser-fit denim styles are trending.
   • Denim-on-denim, also known as the Canadian tuxedo, is making a comeback.
   • '90s and '00s fashion influences, such as denim maxi skirts, are popular.
   • Relaxed silhouettes are favored over skinny jeans.

3. Sources:
   • www.glamour.com
   • www.whowhatwear.com


In [95]:
final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= query_2) #or use: user_prompt_without_context.format(query_2)
additional_options = {"temperature": 0,}

response_2_2, response_full_2_2 = response_generator(openai_client, chat_model=chat_name, prompts=final_prompt, options=additional_options)

cost_eur_2_2 = calculate_total_cost(response_usage= response_full.usage,deployment_name= chat_name)
print(f'Query Completion Total Cost is: {cost_eur_2_2} eur')

Query Completion Total Cost is: 0.00414349086 eur


In [96]:
print(response_2_2)

In 2023, denim fashion trends, as highlighted by Glamour Magazine and Who What Wear, focus on a mix of nostalgic and modern styles. Key trends include:

1. **Baggy and Relaxed Fits**: Loose-fitting jeans, reminiscent of the 90s and early 2000s, are making a strong comeback. These styles prioritize comfort and a laid-back aesthetic.

2. **Low-Rise Jeans**: The low-rise trend is resurging, appealing to those who favor a more daring and retro look.

3. **Cargo and Utility Styles**: Denim with cargo pockets and utility-inspired details are popular, blending functionality with fashion.

4. **Denim Maxi Skirts**: Long denim skirts are trending, offering a versatile and chic alternative to jeans.

5. **Patchwork and Distressed Denim**: These styles add a unique, personalized touch to denim pieces, with patchwork designs and distressed details being particularly popular.

6. **Colored and Printed Denim**: Beyond traditional blue, colored and printed denim options are gaining traction, allowing